In [91]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [92]:
# Loading all data bases 

branchements = pd.read_csv('../data_SUEZ/branchements.csv')
canalisations = pd.read_csv('../data_SUEZ/canalisations.csv')
evenements_train = pd.read_csv('../data_SUEZ/evenements_train.csv')
meteo_AlpesMaritimes = pd.read_csv('../data_SUEZ/meteo_AlpesMaritimes_pre_traite.csv')
nb_ev_train = pd.read_csv('../data_SUEZ/nb_ev_train.csv')
permut_train = pd.read_csv('../data_SUEZ/permut_train.csv')
stations_AlpesMaritimes = pd.read_csv('../data_SUEZ/stations_AlpesMaritimes.csv')

# Preprocessing

## Jointure

In [93]:
# Renaming columns

evenements_train.columns= ['GID_fuite', 'CODINSEE_fuite', 'date_fuite', 'GID_pose', 'COMPO_P', 'geometry_fuite']
branchements.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']
canalisations.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']

In [94]:
evenements_train['fuite'] = 1

In [95]:
# Différencier les GID des branchements et des canalisations

evenements_train['ajout_branch'] = '_b'
evenements_train['ajout_can'] = '_c'
evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'ajout_branch']
evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'ajout_can']
evenements_train.drop('ajout_branch', axis=1, inplace=True)
evenements_train.drop('ajout_can', axis=1, inplace=True)

branchements['ajout_branch'] = '_b'
branchements.GID_pose = branchements.GID_pose.map(str) + branchements.ajout_branch
branchements.drop('ajout_branch', axis=1, inplace=True)

canalisations['ajout_can'] = '_c'
canalisations.GID_pose = canalisations.GID_pose.map(str) + canalisations.ajout_can
canalisations.drop('ajout_can', axis=1, inplace=True)

In [96]:
# Merge elements on evenements (creation BDD complète)
branchements['COMPO_P'] = 'E_BRANCH'
canalisations['COMPO_P'] = 'E_TRONCO'
evenements_train.drop('COMPO_P', axis = 1, inplace = True)
elements = pd.concat([canalisations, branchements])
data = pd.merge(elements, evenements_train, on = 'GID_pose', how = 'left')

## Feature engineering

In [97]:
# Utiliser le code INSEE de la fuite quand on l'a plutot que celui de la pose
data.ix[~data.CODINSEE_fuite.isnull(), 'CODINSEE_pose'] = data.ix[~data.CODINSEE_fuite.isnull(), 'CODINSEE_fuite']
data.drop('CODINSEE_fuite', axis = 1, inplace = True)

In [98]:
# Convertir âge

def to_day(x):
    if (str(x) == 'NaT'):
        return np.nan
    else:
        return x.days

data['age'] = pd.to_datetime('2016-12-31') - pd.to_datetime(data['date_pose'])
data.age = data.age.map(to_day)

In [99]:
data.ix[data.fuite.isnull(), 'fuite'] = 0

In [100]:
# Encode categorical variables

#le = LabelEncoder()
#mapping = dict()
#for col, dtype in zip(data.columns, data.dtypes):
#    if dtype == 'object':
#        data[col] = data[col].apply(lambda s: str(s))
#        # Replace 0 and NaNs with unique label : 'None'
#        data[col] = data[col].where(~data[col].isin(['0', 'nan']), 'None')
#        data[col] = le.fit_transform(data[col])
#        mapping[col] = dict(zip(le.inverse_transform(data[col].unique()), data[col].unique()))

In [101]:
# Replace NaNs with 0
# data = data.apply(lambda x: x.fillna(x.mean()),axis=0)

# Training

## Séparation submit et jeu d'entrainement

In [102]:
# Séparer le dataset d'entrainment du dataset de test
test_insee = [6003, 6004, 6016, 6030, 6070, 6095, 6108, 6112, 6116, 6138, 6152]
data_known = data[~data['CODINSEE_pose'].isin(test_insee)]
submit = data[data['CODINSEE_pose'].isin(test_insee)]

In [103]:
submit = submit.reset_index().ix[:,1:18]
submit_CODINSEE = submit.CODINSEE_pose

In [104]:
# Remove unnecessary columns
fuite_columns = ['GID_fuite', 'date_fuite', 'geometry_fuite']
id_columns = ['GID_pose', 'CODINSEE_pose']
a_predir = ['fuite']
autres = ['GEORIENT', 'date_pose', 'geometry_pose', 'ETAGE', 'DIAMINT', 'DIAMEXT']
mask = id_columns + fuite_columns + a_predir + autres  # geometry_pose pas encore exploitable

X = data_known.drop(mask, axis = 1)
y = data_known['fuite']

X_submit = submit.drop(mask, axis = 1)

## Random Forest

In [105]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(X,y)

ValueError: could not convert string to float: E_BRANCH

In [106]:
y_submit = clf.predict(X_submit)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

# Submitting

In [107]:
def export_submission(mypred):
    mypred = pd.DataFrame(mypred)
    
    submit['pred'] = mypred
    submit['CODEINSEE'] = submit_CODINSEE
    
    mysubmission = pd.DataFrame({'count' : submit.groupby( [ 'CODEINSEE'] )['pred'].sum()}).reset_index()
    mysubmission.columns = ['Codinsee', 'nb_ev'] 
    mysubmission.to_csv('nb_ev_train.csv', sep=',', index=False)

    mysub_rank = mysubmission.sort_values(by = 'nb_ev', ascending=False).ix[:,0]
    mysub_rank.to_csv('permut_train.csv', sep=',', index=False)

In [108]:
export_submission(y_submit)

NameError: name 'y_submit' is not defined

# Metric

In [131]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_actual, y_predicted):
    return (200 - sqrt(mean_squared_error(y_actual, y_predicted)))/200

# Aggrégé par communes

In [135]:
communes = pd.DataFrame(data.CODINSEE_pose.unique())
communes.columns = ['CODINSEE']
communes = pd.DataFrame(communes.sort_values(by = 'CODINSEE').reset_index().ix[:,1])

In [136]:
# Create #canalisation, #fuites

ncan = dict(data.CODINSEE_pose.value_counts())
nfuites = dict(data.ix[data.fuite==1, 'CODINSEE_pose'].value_counts())
communes['ncan'] = communes.CODINSEE.map(lambda x : ncan[x])
communes['nfuites'] = communes.CODINSEE.map(lambda x : nfuites.get(x,0))

# Create longueur total elt
communes['longueur_br'] = branchements.groupby( [ 'CODINSEE_pose'] ).sum().ix[:,4].reset_index().ix[:,1]
communes['longueur_ca'] = canalisations.groupby( [ 'CODINSEE_pose'] ).sum().ix[:,4].reset_index().ix[:,1]

# Create age moyen éléments
hello = pd.DataFrame(data.groupby( [ 'COMPO_P', 'CODINSEE_pose'])['age'].mean())
hello = hello.reset_index()

hello_br = hello.ix[0:15,:]
hello_br.columns = ['COMPO_P', 'CODINSEE', 'age']
test_br = pd.merge(communes, hello_br, on = 'CODINSEE', how = 'left')
communes['age_br'] = test_br['age']

hello_ca = hello.ix[16:36,:]
hello_ca.columns = ['COMPO_P', 'CODINSEE', 'age']
test_ca = pd.merge(communes, hello_ca, on = 'CODINSEE', how = 'left')
communes['age_ca'] = test_ca['age']

In [137]:
# Split submission and training data

communes_test = communes[communes.CODINSEE.isin(test_insee)]
communes_train = communes[~communes.CODINSEE.isin(test_insee)]
Xtrain = communes_train.drop('nfuites', axis = 1)
ytrain = communes_train['nfuites']
Xtest = communes_test.drop('nfuites', axis = 1)
ytest = communes_test['nfuites']

In [138]:
# plt.plot(communes_train.ncan, communes_train.nfuites, 'ro')

In [139]:
lm = LinearRegression()
lm.fit(Xtrain[['CODINSEE','ncan']], ytrain)
ypred = lm.predict(Xtest[['CODINSEE','ncan']])

In [140]:
rmse(ytrain, lm.predict(Xtrain[['CODINSEE','ncan']]))

0.7830342881803984

In [141]:
def export_submission_commune(mypred, Xtest):

    mysubmission = pd.DataFrame(Xtest.CODINSEE)
    mysubmission = mysubmission.reset_index().drop('index', axis = 1)
    mysubmission['nb_ev'] = pd.Series(ypred)
    mysubmission.columns = ['Codinsee', 'nb_ev'] 
    mysubmission.to_csv('nb_ev_train.csv', sep=',', index=False)

    mysub_rank = mysubmission.sort_values(by = 'nb_ev', ascending=False).ix[:,0]
    mysub_rank.to_csv('permut_train.csv', sep=',', index=False)

In [142]:
export_submission_commune(ypred, Xtest)